In [1]:
import torch
import pickle
import tarfile
from tensorflow.keras import layers, models, losses
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import softmax
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def unpickle(file, test_or_data, batch_number):
    with tarfile.open(file, 'r:gz') as tar:
        tar.extractall()
    
    if test_or_data == 'data':
        extracted_file = f'cifar-10-batches-py/{test_or_data}_batch_{batch_number}' 
    if test_or_data == 'test':
        extracted_file = f'cifar-10-batches-py/{test_or_data}_batch'
    
    with open(extracted_file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    
    return data_dict

def convert_flat_to_matrix(image_flat):
    image = np.zeros((32, 32, 3), dtype=np.uint8)
    image[:, :, 0] = image_flat[0:1024].reshape(32, 32)  # Red channel
    image[:, :, 1] = image_flat[1024:2048].reshape(32, 32)  # Green channel
    image[:, :, 2] = image_flat[2048:3072].reshape(32, 32)  # Blue channel
    return image

def prepare_training_data():
    train_data = []
    train_labels = []
    for i in range(1, 6):
        data_batch = unpickle('cifar-10-python.tar.gz', 'data', i)
        for image_flat, label in zip(data_batch[b'data'], data_batch[b'labels']):
            image = convert_flat_to_matrix(image_flat)
            train_data.append(image / 255.0)
            train_labels.append(label)
    return np.array(train_data), np.array(train_labels)

def prepare_test_data():
    test_data = []
    test_labels = []
    data_batch = unpickle('cifar-10-python.tar.gz', 'test', 0)
    for image_flat, label in zip(data_batch[b'data'], data_batch[b'labels']):
        image = convert_flat_to_matrix(image_flat)
        test_data.append(image / 255.0)
        test_labels.append(label)
    return np.array(test_data), np.array(test_labels)

def initialize_cnn(pooling_type, conv_width):
    input_shape = (32, 32, 3)

    # Create a Sequential model
    model = models.Sequential()
    
    # Add the input layer with the defined input shape
    model.add(layers.Input(shape=input_shape))
    
    # Add convolutional layers
    model.add(layers.Conv2D(32, (conv_width, conv_width), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(pooling_type((2, 2)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(64, (conv_width, conv_width), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(pooling_type((2, 2)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(128, (conv_width, conv_width), activation='relu'))
    
    # Flatten and add dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(10))
    return model

def map_function_to_str(function):
    if function == layers.MaxPooling2D:
        return 'max_pooling'
    if function == layers.AveragePooling2D:
        return 'average_pooling'
    raise "Unsupported function"

In [3]:
x_train, y_train = prepare_training_data()
x_test, y_test = prepare_test_data()

In [4]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

datagen.fit(x_train)

In [5]:
model = initialize_cnn(layers.MaxPooling2D, 3)

In [15]:
results_df = pd.DataFrame(columns = ['epochs', 'train_loss', 'train_acc', 'test_loss', 'test_acc'])
results_df.set_index('epochs', inplace=True)

model.compile(optimizer='adam',
              loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
batch_size = 32
train_data_gen = datagen.flow(x_train, y_train, batch_size=batch_size)
steps_per_epoch = x_train.shape[0]//batch_size
history = model.fit(train_data_gen, epochs=50, batch_size = 64, validation_data = (x_test, y_test))


Epoch 1/50


C:\Users\jkuta\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 44s 24ms/step - accuracy: 0.7110 - loss: 0.8509 - val_accuracy: 0.8000 - val_loss: 0.5961
Epoch 2/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.7106 - loss: 0.8393 - val_accuracy: 0.7941 - val_loss: 0.6179
Epoch 3/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 38s 24ms/step - accuracy: 0.7159 - loss: 0.8354 - val_accuracy: 0.7873 - val_loss: 0.6398
Epoch 4/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step - accuracy: 0.7153 - loss: 0.8328 - val_accuracy: 0.7707 - val_loss: 0.7057
Epoch 5/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step - accuracy: 0.7148 - loss: 0.8379 - val_accuracy: 0.7935 - val_loss: 0.6196
Epoch 6/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 24ms/step - accuracy: 0.7169 - loss: 0.8359 - val_accuracy: 0.7224 - val_loss: 0.8692
Epoch 7/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 38s 24ms/step - accuracy: 0.7094 - loss: 0.8409 - val_accuracy: 0.7496 - val_loss: 0.8103
Epoch 8/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 38s 24ms/step - accuracy: 0.7146 - loss: 0.82

In [ ]:
# Example to make sure I understand how these images work
data = unpickle('cifar-10-python.tar.gz', 'test', 1)
image_flat = data[b'data'][0]
image = convert_flat_to_matrix(image_flat)

# Display the image
plt.imshow(image)
plt.title(f'Image {0}')
plt.show()

data[b'labels'][0]

In [ ]:
for key, model in model_dict.items():
    predictions = model.predict(x_test)

    # Example output shapes
    print('Shape of predictions:', predictions.shape)  # Shape will depend on your model and task
    
    # Example of accessing predictions for the first sample
    first_prediction = predictions[0]
    print('First prediction:', first_prediction)

In [ ]:
softmax(first_prediction)